In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy


# Import Required Libraries & Data Loading

In [2]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
print(df['TEXT_COLUMN_NAME'])

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: TEXT_COLUMN_NAME, Length: 50000, dtype: object


In [7]:
df['totalwords'] = df['TEXT_COLUMN_NAME'].str.split().str.len()
print(df['totalwords'])

0        307
1        162
2        166
3        138
4        230
        ... 
49995    194
49996    112
49997    230
49998    212
49999    129
Name: totalwords, Length: 50000, dtype: int64


In [8]:
from collections import Counter

print("Zero-length reviews: {}".format(df['totalwords'].min()))
print("Maximum review length: {}".format(df['totalwords'].max()))

Zero-length reviews: 4
Maximum review length: 2470


In [9]:
# def pad_features(reviews_ints, seq_length):
#     ''' Return features of review_ints, where each review is padded with 0's 
#         or truncated to the input seq_length.
#     '''
#     ## getting the correct rows x cols shape
#     features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
#     ## for each review, I grab that review
#     for i, row in enumerate(reviews_ints):
#       features[i, -len(row):] = np.array(row)[:seq_length]
    
#     return features

In [ ]:
# seq_length = 200

# features = pad_features(reviews_ints, seq_length=seq_length)

# ## test statements - do not change - ##
# assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
# assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# # print first 10 values of the first 30 batches 
# print(features[:30,:10])

In [10]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
spacy.load("en_core_web_sm")


In [62]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.01
BATCH_SIZE = 50
NUM_EPOCHS = 30
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 256
HIDDEN_DIM = 256
NUM_CLASSES = 2

In [2]:
!pip install -U torch==1.8.0 torchtext==0.9.0

     |████████████████████████████████| 735.5 MB 14 kB/s 
     |████████████████████████████████| 7.1 MB 35.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


# Text & label Preparation

In [12]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', fix_length=256)

In [13]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [14]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME,totalwords
0,One of the other reviewers has mentioned that ...,1,307
1,A wonderful little production. <br /><br />The...,1,162
2,I thought this was a wonderful way to spend ti...,1,166
3,Basically there's a family where a little boy ...,0,138
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,230


In [16]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = "moviedata.csv",
                    format = "CSV",
                    skip_header = True,
                    fields =  [('TEXT_COLUMN_NAME', TEXT),('LABEL_COLUMN_NAME',LABEL)]
)

# Data Split

In [17]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [18]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [19]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': '1'}


In [20]:
from typing import Text
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

print(TEXT.vocab)

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [61]:
# Print the most common words: Use the most_common method of the TEXT vocabulary

most_common_words = TEXT.vocab.freqs.most_common(30)
print(most_common_words)

[('the', 390972), (',', 369444), ('.', 318719), ('a', 210502), ('and', 210006), ('of', 194658), ('to', 180163), ('is', 145895), ('in', 118266), ('I', 105681), ('it', 103588), ('that', 93995), ('"', 85530), ("'s", 83147), ('this', 81771), ('-', 70411), ('/><br', 68787), ('was', 67372), ('as', 57734), ('movie', 57571), ('with', 57543), ('for', 56557), ('film', 52382), ('The', 51249), ('but', 46574), ('(', 44823), (')', 44506), ('on', 44484), ("n't", 44474), ('you', 42196)]


In [26]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


In [21]:
# from string import punctuation

# all_text = ''.join([c for c in str(TEXT.vocab.itos).replace(',', ' ') if c not in punctuation])

# reviews_ints = []
# for review in all_text:
#   reviews_ints.append([TEXT.vocab[word] for word in review.split()])

# # print tokens in first review
# print('Tokenized review: \n', reviews_ints[:1])

# Data Preparation for Batch wise Implimentation

In [63]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_sizes = (50,50,50), ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = False, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

In [64]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([256, 50])
Target vector size: torch.Size([50])

Valid:
Text matrix size: torch.Size([256, 50])
Target vector size: torch.Size([50])

Test:
Text matrix size: torch.Size([256, 50])
Target vector size: torch.Size([50])


In [29]:
# train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
#         [train_data, val_data, test_data], ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
#         batch_size = 5, ### ADD YOUR BATCH SIZE HERE ###
#         sort_within_batch = 5, ### ADD YOUR SORT WITHIN BATCH HERE ### 
#         sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
#         device = DEVICE
#     )

# Model Building

In [41]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, drop_prob=0.5):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        print('Model')
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

    
        # embedding and LSTM layers
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        
        #lstm
        self.lstm = torch.nn.LSTM(input_size=embedding_dim,hidden_size=hidden_dim)
    
        # linear and sigmoid layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        self.sig = torch.nn.Sigmoid()
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        
        batch_size = text.size(0)
        
        embeds = self.embedding(text) 

        lstm_out, hidden = self.lstm(embeds)

        # print(len(hidden[0]))

        hidden = hidden[0].squeeze(0)

        out = self.fc(hidden)
        
        output = self.sig(out)
      
        
        ### END YOUR CODE ###
        return output

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        weight = next(self.parameters()).data
        
        hidden = (weight.new(2, batch_size, self.hidden_dim).zero_(),
                   weight.new(2, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [66]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab), ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim=50, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=256, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=1  ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Model


# Define Accuracy

In [67]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            # print(logits)
            _, predicted_labels = torch.max(logits,1)
            # print(predicted_labels)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [70]:
start_time = time.time()
criterion = torch.nn.BCELoss()

# NUM_EPOCHS
for epoch in range(NUM_EPOCHS):
    model.train()

    # h = model.init_hidden(BATCH_SIZE)

    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        
        model.zero_grad()
        output = model(text).squeeze(1)
        # print(output.shape)
        
        loss = criterion(output, labels.float())
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/030 | Batch 000/680 | Loss: 0.6799
Epoch: 001/030 | Batch 050/680 | Loss: 0.6851
Epoch: 001/030 | Batch 100/680 | Loss: 0.6611
Epoch: 001/030 | Batch 150/680 | Loss: 0.7979
Epoch: 001/030 | Batch 200/680 | Loss: 0.5880
Epoch: 001/030 | Batch 250/680 | Loss: 0.5773
Epoch: 001/030 | Batch 300/680 | Loss: 0.5294
Epoch: 001/030 | Batch 350/680 | Loss: 0.5873
Epoch: 001/030 | Batch 400/680 | Loss: 0.5678
Epoch: 001/030 | Batch 450/680 | Loss: 0.6744
Epoch: 001/030 | Batch 500/680 | Loss: 0.4251
Epoch: 001/030 | Batch 550/680 | Loss: 0.3697
Epoch: 001/030 | Batch 600/680 | Loss: 0.6723
Epoch: 001/030 | Batch 650/680 | Loss: 0.4342
training accuracy: 50.09%
valid accuracy: 50.17%
Time elapsed: 1.07 min
Epoch: 002/030 | Batch 000/680 | Loss: 0.5500
Epoch: 002/030 | Batch 050/680 | Loss: 0.4689
Epoch: 002/030 | Batch 100/680 | Loss: 0.3067
Epoch: 002/030 | Batch 150/680 | Loss: 0.3346
Epoch: 002/030 | Batch 200/680 | Loss: 0.4321
Epoch: 002/030 | Batch 250/680 | Loss: 0.4983
Epoch: 0

# Model Testing

In [51]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


1.0

In [52]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


1.0